# On Classifier Evaluation

In [64]:
from sklearn.metrics import confusion_matrix
import numpy as np
from sklearn.utils import resample
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

## Confusion Matrix

For classification problems, the target is a categorical variable. This means that we can simply count the number of times that our model predicts the correct category and the number of times that it predicts something else.

We can visualize this by means of a **confusion matrix**, which displays counts of correct and incorrect predictions. We'll explore this below. There are [many ways](https://docs.google.com/document/d/12BkMOXt5-iMzw_y_zFLZNkPlgAy1rj1t9gJ6LXi2bqQ/edit?usp=sharing) of evaluating a classification model, but most derive from the confusion matrix.

## Lottery Number Prediction

Suppose I want to train a model to predict whether a string of six numbers (a "ticket") would win the lottery or not. What sort of data might I use?

### Scenario 1

I gather all the winning tickets from the last 10000 days or so. So I have one column for the tickets themselves, and a Boolean column indicating whether the ticket was a winner or not.

Now if all the tickets on which my model trains are *winning* tickets, then it would predict every ticket to win! Suppose I test the model on a set of 1000 tickets, and suppose that there is exactly one winning ticket among those 1000. My model will always predict the ticket to win. Let's think about what the confusion matrix will look like.

In [65]:
# Setting up the true values

y_true = np.zeros(1000)
y_true[500] = 1

y_true

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [66]:
# Setting up the predictions

y_pred = np.ones(1000)

y_pred

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

In [67]:
# Defining the confusion matrix

cm_1 = confusion_matrix(y_true, y_pred)

The confusion matrix should tell us that we have 999 false positives (999 losing tickets predicted to win) and 1 true positive (1 winning ticket predicted to win):

In [68]:
cm_1

array([[  0, 999],
       [  0,   1]])

Notice the way that sklearn displays its confusion matrix: The rows are \['actually false', 'actually true'\]; the columns are \['predicted false', 'predicted true'\].

So it displays:

$\begin{bmatrix}
TN & FP \\
FN & TP
\end{bmatrix}$

In [69]:
tn = cm_1[0, 0]
fp = cm_1[0, 1]
fn = cm_1[1, 0]
tp = cm_1[1, 1]

Let's see if we can calculate some of our metrics for this matrix.

**Accuracy** = $\frac{TP + TN}{TP + TN + FP + FN}$

In words: How often did my model get the right answer?

In [70]:
# Accuracy

print(f'Accuracy: ({tp} + {tn}) / ({tp} + {tn} + {fp} + {fn}) = 1/1000 = {1/1000}')

Accuracy: (1 + 0) / (1 + 0 + 999 + 0) = 1/1000 = 0.001


**Recall** = $\frac{TP}{TP + FN}$

In words: How often did my model correctly predict winning tickets?

In [71]:
# Recall

print(f'Recall: {tp} / ({tp} + {fn}) = 1/1 = {1}')

Recall: 1 / (1 + 0) = 1/1 = 1


**Precision** = $\frac{TP}{TP + FP}$

In words: How often was my model's prediction of 'winner' correct?

In [72]:
# Precision

print(f'Precision: {tp} / ({tp} + {fp}) = 1 / 1000 = {1/1000}')

Precision: 1 / (1 + 999) = 1 / 1000 = 0.001


**F-1 Score** = $\frac{2PrRc}{Pr + Rc}$ = $\frac{2TP}{2TP + FP + FN}$

In [73]:
# F-1 Score

print(f'F-1 Score: (2)({tp}) / ((2)({tp}) + {fp} + {fn}) = 2 / 1001 = {round(2/1001, 3)}')

F-1 Score: (2)(1) / ((2)(1) + 999 + 0) = 2 / 1001 = 0.002


### Scenario 2

Well, my recall was good, but everything else I measured was terrible! Can I do better?

This time I'll train my model in a much different way. I'll give it the tickets of 10000 people who played the lottery yesterday. Suppose that there is one winning ticket and 9999 losers. Now I test the model on the same 1000 tickets from before in Scenario 1.

This time my model will almost always predict a ticket to lose. Suppose that, in the 1000 predictions, it makes only one prediction of a winner, and suppose that this prediction is wrong.

#### Exercise: Set up the confusion matrix and compute our metrics for this new scenario.

In [74]:
y_true2 = y_true
y_pred2 = np.zeros(1000)
y_pred2[0] = 1


In [75]:
cm2 = confusion_matrix(y_true2,  y_pred2)
cm2

array([[998,   1],
       [  1,   0]])

In [78]:
print('acc: 998/1000')
print('recall: 0')
print('prec: 0')

acc: 998/1000
recall: 0
prec: 0


### Lessons

The last classifier had a really high accuracy, but everything else was terrible.

It occurs to me now that if I'm really going to get a good model running, I'm going to have to have good numbers of **both winning and losing  tickets**. The first model didn't see enough losers, and the second model didn't see enough winners.

Of course, I could just be more careful about my data collection, but there are often easier fixes. One of the most effective strategies is **oversampling the minority class**. That is, I give myself more data points than I really have. I could achieve this either by [bootstrapping](https://scikit-learn.org/stable/modules/generated/sklearn.utils.resample.html) or by generating some data that is fake but close to actual data. The latter is the idea behind [SMOTE](https://imbalanced-learn.readthedocs.io/en/stable/generated/imblearn.over_sampling.SMOTE.html).

## Breast Cancer Prediction

Let's see what results we get on scikit-learn's breast cancer dataset:

### Loading the Data

In [79]:
preds, target = load_breast_cancer(return_X_y=True)

One way to measure the relative size of our classes:

In [85]:
import pandas as pd
pd.DataFrame(preds)

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


In [82]:
target[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [80]:
sum(target) / len(target)

0.6274165202108963

Let's assume that this is close enough to "balanced"!

### Splitting and Scaling:

In [94]:
X_train, X_test, y_train, y_test = train_test_split(preds, target)

In [95]:
ss = StandardScaler()

ss.fit(X_train)

X_train_sc = ss.transform(X_train)
X_test_sc = ss.transform(X_test)

### Model Fitting

In [96]:
logreg = LogisticRegression(solver='lbfgs', max_iter=10000)

logreg.fit(X_train_sc, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=10000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

### Confusion Matrix

In [105]:
cancer_cm = confusion_matrix(y_test, logreg.predict(X_test_sc))
cancer_cm

array([[52,  4],
       [ 0, 87]])

### Interpretation

#### Exercise: What are the precision, recall, and accuracy of our model?

In [106]:
acc = (52+87)/(52+4+0+87)
prec = 87/(87+4)
rec = 87/(87+0)
print('acc: ', acc, 'prec: ', prec, 'rec: ', rec)

acc:  0.972027972027972 prec:  0.9560439560439561 rec:  1.0


## Which Metric Should I Care About?

Well, it depends.

### General Lessons

First, let's make some general observations about the metrics we've so far defined.

Accuracy:
- Pro: Takes into account both false positives and false negatives.
- Con: Can be misleadingly high when there is a significant class imbalance. (A lottery-ticket predictor that *always* predicts a loser will be highly accurate.)

Recall:
- Pro: Highly sensitive to false negatives.
- Con: No sensitivity to false positives.

Precision:
- Pro: Highly sensitive to false positives.
- Con: No sensitivity to false negatives.

F-1 Score:
- Harmonic mean of recall and precision.

### Cost

The nature of your business problem will help you determine which metric matters.

Sometimes false positives are much worse than false negatives: Arguably, a model that compares a sample of crime-scene DNA with the DNA in a city's database of its citizens presents one such case. Here a false positive would mean falsely identifying someone as having been present at a crime scene, whereas a false negative would mean only that we fail to identify someone who really was present at the crime scene as such.

On the other hand, consider a model that inputs X-ray images and predicts the presence of cancer. Here false negatives are surely worse than false positives: A false positive means only that someone without cancer is misdiagnosed as having it, while a false negative means that someone with cancer is misdiagnosed as *not* having it.

### Strategies

There are a couple of strategies for trying to quantify these important differences in value between false positives and false negatives.

#### Strategy 1: Use a cost matrix.

One might assign different weights to the costs associated with false positives and false negatives. (We'll standardly assume that the costs associated with *true* positives and negatives are negligible.)

**Example**. Suppose we are in the DNA prediction scenario above. Then we might construct the following cost matrix:

In [107]:
cost = np.array([[0, 10], [3, 0]])
cost

array([[ 0, 10],
       [ 3,  0]])

This cost matrix will allow us to compare models if we have access to those models' rates of false positives and false negatives, i.e. if we have access to the models' confusion matrices!

**Problem**. Given the cost matrix above and the confusion matrices below, which model should we go with?

In [108]:
conf1, conf2 = np.array([[100, 10], [30, 300]]), np.array([[120, 20], [0, 300]])

print(conf1, 2*'\n', conf2)

[[100  10]
 [ 30 300]] 

 [[120  20]
 [  0 300]]


In [111]:
cost1 = 10 * 10 + 3 * 30
print(cost1)
cost2 = 10 * 20
print(cost2)

190
200
